## Reserve and configure resources on Chameleon

## Create lease and reserve resources


In [74]:
import json
import os
import chi
import chi.ssh
import chi.network
from datetime import datetime, timedelta
from dateutil import tz

In [75]:
#Config with your project and site
project = os.getenv("OS_PROJECT_NAME")
chi.use_site('CHI@UC')
chi.set('project_name', project)

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


In [76]:
# Name resources with your username for easier identification
username = os.getenv("USER")
suffix = username + "_k8s-ml"
server_prefix = "node" 
network_name = "net-" + suffix
subnet_name = "subnet-" + suffix
router_name = "router-" + suffix
lease_name = "lease-" + suffix

In [77]:
#Server attributes
image_name='CC-Ubuntu18.04'
node_type="compute_cascadelake_r"
server_count=3
physical_network="physnet1"
subnet_cidr = '192.168.1.0/24'

#### Create the Network

In [78]:
# Create a network out of provider network physnet1
network = chi.network.create_network(network_name, provider='physnet1')
network_id = network['id']

print(f'Network ID: {network_id}')

Network ID: 4579959b-5b2f-4f2e-bc02-502884cc1e13


#### Configure the Network


In [79]:
subnet = chi.network.create_subnet(subnet_name, network_id, cidr=subnet_cidr, gateway_ip=None)
router = chi.network.create_router(router_name, gw_network_name='public')
chi.network.add_subnet_to_router(router['id'], subnet['id'])

{'id': '904b75e0-5007-40f2-9be6-792f8af141ae',
 'tenant_id': '49f47d2c64e64937840c3f7c663a37b2',
 'port_id': 'e1753f30-4597-4907-9eed-7ed3b230ff99',
 'network_id': '4579959b-5b2f-4f2e-bc02-502884cc1e13',
 'subnet_id': '3d74460a-099a-48ac-96fb-c04d045da512',
 'subnet_ids': ['3d74460a-099a-48ac-96fb-c04d045da512']}

#### Create a lease with network and 3 servers

In [80]:
import chi.lease

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=3)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
chi.lease.add_node_reservation(reservation_list, count=server_count, node_type=node_type)
chi.lease.add_fip_reservation(reservation_list, count=server_count)

# Create the lease
lease = chi.lease.create_lease(lease_name, 
                               start_date=start_date,
                               end_date=end_date,
                               reservations=reservation_list)

lease_id = lease["id"]

chi.lease.wait_for_active(lease_id)

{'created_at': '2023-05-22 22:24:16',
 'updated_at': '2023-05-22 22:25:07',
 'id': '7bc6d3c1-085d-44a9-860a-3301b4d07697',
 'name': 'lease-cp3793_nyu_edu_k8s-ml',
 'user_id': '50de6f77f6d1941774ecf322c9f0ad2a3e2c128f4707c3278d98fd4a98b86d85',
 'project_id': '49f47d2c64e64937840c3f7c663a37b2',
 'start_date': '2023-05-22T22:25:00.000000',
 'end_date': '2023-05-25T22:24:00.000000',
 'trust_id': '3b2edb8afa974d9fb0adfde374f7517a',
 'status': 'ACTIVE',
 'degraded': False,
 'reservations': [{'created_at': '2023-05-22 22:24:17',
   'updated_at': '2023-05-22 22:25:05',
   'id': '50a46f7a-31f6-4f66-9b31-c343f8f8d5e2',
   'lease_id': '7bc6d3c1-085d-44a9-860a-3301b4d07697',
   'resource_id': 'fba00cec-1a94-4859-88ea-8562f33b3bfd',
   'resource_type': 'virtual:floatingip',
   'status': 'active',
   'missing_resources': False,
   'resources_changed': False,
   'network_id': '44b38c44-2a42-4b6d-b129-6c8f1b2a1375',
   'amount': 3,
   'required_floatingips': []},
  {'created_at': '2023-05-22 22:24:16'

#### Get the Reservations

In [81]:
compute_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
floatingip_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'virtual:floatingip'][0]['id']

print(f"compute_reservation_id: {compute_reservation_id}")
print(f"floatingip_reservation_id: {floatingip_reservation_id}")

compute_reservation_id: beb33eab-2a6c-44f3-a134-d81f79dd1e74
floatingip_reservation_id: 50a46f7a-31f6-4f66-9b31-c343f8f8d5e2


#### Start the Server

In [82]:
#create the server
servers = []
for i in range(server_count):
    servers.append(chi.server.create_server(server_prefix+f"-{i}-" + suffix, 
                                  compute_reservation_id,
                                  network_id=network_id,
                                  nics=[{"net-id": network_id,"v4-fixed-ip": f"192.168.1.1{i}" }],
                                  image_name=image_name,
                                  count=1))


#### Associate the floating IPs

In [83]:
for j,i in enumerate(servers):
    chi.server.wait_for_active(i.id)
    floating_ip = chi.server.associate_floating_ip(i.id)
    nodes[j] = floating_ip

    print(f'Floating IP of node_{j}: {floating_ip}')

Floating IP of node_0: 192.5.86.238
Floating IP of node_1: 192.5.87.207
Floating IP of node_2: 192.5.87.224


#### Wait for SSH access to all servers

In [84]:
for node_ip in nodes.values():
    chi.server.wait_for_tcp(node_ip, port=22)

### Configuring the servers

#### Add your public key

In [85]:
from chi.ssh import Remote
for node_ip in nodes.values():
    remote = Remote(node_ip)
    nova=chi.clients.nova()
    [kp.name for kp in nova.keypairs.list()]
    keypair_name = 'id_rsa_chameleon' # The name of the key they uploaded 
    public_key = nova.keypairs.get(keypair_name).public_key # Contents of the public key
    remote.run(f"echo {public_key} >> ~/.ssh/authorized_keys")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 192.5.86.238: b'1225befe30e61f410a0e205c877f0fd2'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 192.5.87.207: b'c18b80cb527a69de856a32a66f02f61d'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 192.5.87.224: b'2f8eb43d4d0447015e93ae1b3c617562'
  warnings.warn(


#### Install keys for SSH access between all servers

In [87]:
for node_ip in nodes.values():
    remote = chi.ssh.Remote(node_ip)
    remote.run('ssh-keygen -t rsa -b 4096 -f ~/.ssh/id_rsa -q -N ""')
    public_key = remote.run('cat ~/.ssh/id_rsa.pub').tail("stdout")[2:]
    public_key = public_key.replace("\n", "")
    for other_node_ip in nodes.values():
        remote = chi.ssh.Remote(other_node_ip)
        remote.run(f'echo {public_key} >> ~/.ssh/authorized_keys') 

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDSZ4IMHNAxVR/qa/vhG182nQtrF5Qx4GDLZPmFxin1I6DXo/LyilJBr/FtmR6rk7CGCLkNJAba0XtBxpmddtJVc6q1o1Nzmx7aYsheLDRumI3FphgczBdpmNefHgVBpnaweQG7LG2CDkAL9pw+FdjWkdxvpYnSdsImqPccTDlwdeafOFwqxa77tZH31b/752TTEvDkD5+sZAt7Y2Ew569UcH9S8tDUeblXEsjKM3yiQp2leIa+O6L2oRrBcY19VRSwhgSGXk/pJ0KziRQLg4lWG9NHDksFfevoWy7knDdu54ElooKeNF4mgkyNEhRLSiXDGyp3K8ukWiPg7apDA3zf4S67op5h16ekdHL2bxVPcDwftFgYsu0mbK4Lir663W9G+9843oELGUzgRjMsACX9zf8zZQjM3KQh/1DusixwWvhtKdkMbNE0/GTLQQlhzAbUanBM9N7/W0Q/XlE78DeAGazvVKMmgIUpT9M6nyr8iRKS/g1t9MkFGu1fISSPHIuRA3UvEhMkLXUdoJNl9FZ/ZmETV5t0+9guHEinSZEwJzT//fdAk3UpmDeKYo17xnuWKX0FtVt6M2z6ArTvGFYay4LxOU9OyW/44IRZjeBhCH3a8RGprQBsO+VfO1BMtfE3HVsUnIsQ0jZ93Y6ZubO3awRQA8/KWhA2geA9wuu//w== cc@node-0-cp3793-nyu-edu-k8s-ml
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCzmBUF2vaLvzdsXsPIW3NQbhK39UCHmyYTSl9VCl/bvcDtI+lXkhF7O5SMc2O0Ujh1HoxmBvK9S9qUW++AppqWITSLt/ytzuQ5/h6QfOcv8lG+1U0PtDJyYlnTo9/pVbtYWOev+S2fKZ2ihEme5iiLbrGnD2IVlE4QN+Hk9MY1jM11c0RLdzVXuqNVD+Sd0qIqgme4IAnDwVP

#### Store the IP address of each host

In [88]:
physical_ips=[]
for node_ip in nodes.values():
    chi.server.wait_for_tcp(node_ip, port=22)
    remote = chi.ssh.Remote(node_ip)
    physical_ips.append(remote.run('hostname -I').tail("stdout")[2:-1])

192.168.1.10 
192.168.1.11 
192.168.1.12 


#### Use Kubespray to prepare a Kubernetes cluster

In [89]:
ansible_node = nodes[0]
remote = chi.ssh.Remote(ansible_node)
remote.run("sudo apt install virtualenv")
remote.run("virtualenv -p python3 myenv")
remote.run("git clone https://github.com/kubernetes-sigs/kubespray.git")
remote.run("cd kubespray; git checkout release-2.16")
remote.run("source myenv/bin/activate; cd kubespray; pip3 install -r requirements.txt")
remote.run("cd kubespray; cp -rfp inventory/sample inventory/mycluster")
remote.run("rm -rf kubespray/contrib/inventory_builder/inventory.py")
remote.run("rm -rf kubespray/inventory/mycluster/group_vars/k8s_cluster/addons.yml")
remote.run("wget https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/inventory.py -O kubespray/contrib/inventory_builder/inventory.py")
remote.run("wget https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/addons.yml -O kubespray/inventory/mycluster/group_vars/k8s_cluster/addons.yml")
remote.run(f"source myenv/bin/activate; declare -a IPS=({physical_ips[0]} {physical_ips[1]} {physical_ips[2]});"+"cd kubespray; CONFIG_FILE=inventory/mycluster/hosts.yaml python3 contrib/inventory_builder/inventory.py ${IPS[@]}")

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  virtualenv
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 4476 B of archives.
After this operation, 30.7 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu bionic/universe amd64 virtualenv all 15.1.0+ds-1.1 [4476 B]


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 4476 B in 0s (27.4 kB/s)
Selecting previously unselected package virtualenv.
(Reading database ... 63374 files and directories currently installed.)
Preparing to unpack .../virtualenv_15.1.0+ds-1.1_all.deb ...
Unpacking virtualenv (15.1.0+ds-1.1) ...
Setting up virtualenv (15.1.0+ds-1.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Already using interpreter /usr/bin/python3
Using base prefix '/usr'
New python executable in /home/cc/myenv/bin/python3
Also creating executable in /home/cc/myenv/bin/python
Installing setuptools, pkg_resources, pip, wheel...done.


Cloning into 'kubespray'...


Branch 'release-2.16' set up to track remote branch 'release-2.16' from 'origin'.


Switched to a new branch 'release-2.16'


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ansible: filename=ansible-2.9.20-py3-none-any.whl size=16203449 sha256=2cbf0df0774266cf6b1d71e50703e04e88882bc9f2c6502cbdad3ec55ed1b102
  Stored in directory: /home/cc/.cache/pip/wheels/0f/36/b7/907fa25c2228a59030698d2ac557ba5c694b648dede82fb7e1
Successfully built ansible


--2023-05-22 22:55:15--  https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/inventory.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17940 (18K) [text/plain]
Saving to: ‘kubespray/contrib/inventory_builder/inventory.py’

     0K .......... .......                                    100%  147M=0s

2023-05-22 22:55:15 (147 MB/s) - ‘kubespray/contrib/inventory_builder/inventory.py’ saved [17940/17940]

--2023-05-22 22:55:15--  https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/addons.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTT

DEBUG: Adding group all
DEBUG: Adding group kube_control_plane
DEBUG: Adding group kube_node
DEBUG: Adding group etcd
DEBUG: Adding group k8s_cluster
DEBUG: Adding group calico_rr
DEBUG: adding host node-0 to group all
DEBUG: adding host node-1 to group all
DEBUG: adding host node-2 to group all
DEBUG: adding host node-0 to group etcd
DEBUG: adding host node-1 to group etcd
DEBUG: adding host node-2 to group etcd
DEBUG: adding host node-0 to group kube_control_plane
DEBUG: adding host node-1 to group kube_control_plane
DEBUG: adding host node-0 to group kube_node
DEBUG: adding host node-1 to group kube_node
DEBUG: adding host node-2 to group kube_node


<Result cmd='source myenv/bin/activate; declare -a IPS=(192.168.1.10 192.168.1.11 192.168.1.12);cd kubespray; CONFIG_FILE=inventory/mycluster/hosts.yaml python3 contrib/inventory_builder/inventory.py ${IPS[@]}' exited=0>

In [90]:
# need a specific versino of pyOpenSSL
remote.run("source myenv/bin/activate; sudo rm -rf /usr/local/lib/python3.6/dist-packages/OpenSSL")
remote.run("source myenv/bin/activate; sudo rm -rf /usr/local/lib/python3.6/dist-packages/pyOpenSSL-22.1.0.dist-info/")
remote.run("source myenv/bin/activate; sudo pip3 install pyOpenSSL==22.0.0")

  Attempting uninstall: pyOpenSSL
    Found existing installation: pyOpenSSL 17.5.0
    Uninstalling pyOpenSSL-17.5.0:
      Successfully uninstalled pyOpenSSL-17.5.0


<Result cmd='source myenv/bin/activate; sudo pip3 install pyOpenSSL==22.0.0' exited=0>

In [91]:
remote.run("source myenv/bin/activate; cd kubespray; ansible-playbook -i inventory/mycluster/hosts.yaml  --become --become-user=root cluster.yml")


PLAY [localhost] ***************************************************************
Monday 22 May 2023  22:56:36 +0000 (0:00:00.032)       0:00:00.032 ************ 

TASK [Check 2.9.0 <= Ansible version < 2.11.0] *********************************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}
Monday 22 May 2023  22:56:36 +0000 (0:00:00.014)       0:00:00.047 ************ 

TASK [Check that python netaddr is installed] **********************************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}
Monday 22 May 2023  22:56:36 +0000 (0:00:00.013)       0:00:00.061 ************ 

TASK [Check that jinja is not too old (install via pip)] ***********************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}


[WARNING]: Could not match supplied host pattern, ignoring: kube-master
[WARNING]: Could not match supplied host pattern, ignoring: kube-node



PLAY [Add kube-master nodes to kube_control_plane] *****************************
skipping: no hosts matched

PLAY [Add kube-node nodes to kube_node] ****************************************


[WARNING]: Could not match supplied host pattern, ignoring: k8s-cluster


skipping: no hosts matched


...

[WARNING]: flush_handlers task does not support when conditional


Monday 22 May 2023  22:57:08 +0000 (0:00:00.032)       0:00:32.537 ************ 
Monday 22 May 2023  22:57:08 +0000 (0:00:00.031)       0:00:32.569 ************ 

TASK [container-engine/containerd-common : gather os specific variables] *******
ok: [node-0] => (item=/home/cc/kubespray/roles/container-engine/containerd-common/vars/../vars/ubuntu.yml)
ok: [node-1] => (item=/home/cc/kubespray/roles/container-engine/containerd-common/vars/../vars/ubuntu.yml)
ok: [node-2] => (item=/home/cc/kubespray/roles/container-engine/containerd-common/vars/../vars/ubuntu.yml)
Monday 22 May 2023  22:57:08 +0000 (0:00:00.056)       0:00:32.625 ************ 
Monday 22 May 2023  22:57:08 +0000 (0:00:00.042)       0:00:32.668 ************ 
Monday 22 May 2023  22:57:08 +0000 (0:00:00.034)       0:00:32.702 ************ 
Monday 22 May 2023  22:57:08 +0000 (0:00:00.032)       0:00:32.734 ************ 
Monday 22 May 2023  22:57:08 +0000 (0:00:00.031)       0:00:32.766 ************ 
Monday 22 May 2023  22:57:08 +

[WARNING]: noop task does not support when conditional



TASK [download : prep_kubeadm_images | Create kubeadm config] ******************
changed: [node-0]
changed: [node-1]
Monday 22 May 2023  22:57:41 +0000 (0:00:00.373)       0:01:05.662 ************ 

TASK [prep_kubeadm_images | Copy kubeadm binary from download dir to system path] ***
changed: [node-1]
changed: [node-0]
Monday 22 May 2023  22:57:42 +0000 (0:00:00.325)       0:01:05.988 ************ 

TASK [download : prep_kubeadm_images | Set kubeadm binary permissions] *********
ok: [node-0]
ok: [node-1]
Monday 22 May 2023  22:57:42 +0000 (0:00:00.154)       0:01:06.143 ************ 

TASK [download : prep_kubeadm_images | Generate list of required images] *******
ok: [node-0]
Monday 22 May 2023  22:57:42 +0000 (0:00:00.257)       0:01:06.401 ************ 

TASK [download : prep_kubeadm_images | Parse list of images] *******************
ok: [node-0] => (item=k8s.gcr.io/kube-apiserver:v1.20.7)
ok: [node-0] => (item=k8s.gcr.io/kube-controller-manager:v1.20.7)
ok: [node-0] => (item=k8s.g

<Result cmd='source myenv/bin/activate; cd kubespray; ansible-playbook -i inventory/mycluster/hosts.yaml  --become --become-user=root cluster.yml' exited=0>

In [92]:
# give cc user the kube config file
remote = chi.ssh.Remote(nodes[0])
remote.run("sudo cp -R /root/.kube /home/cc/.kube; sudo chown -R cc /home/cc/.kube; sudo chgrp -R cc /home/cc/.kube")

<Result cmd='sudo cp -R /root/.kube /home/cc/.kube; sudo chown -R cc /home/cc/.kube; sudo chgrp -R cc /home/cc/.kube' exited=0>

In [94]:
# enable external access to the port on which we will deploy the service
for i in nodes.keys():
    remote = chi.ssh.Remote(nodes[i])
    remote.run("sudo ufw allow 32000")

Rule added
Rule added (v6)
Rule added
Rule added (v6)
Rule added
Rule added (v6)


In [95]:
# also make sure docker can run without sudo 
# start a new SSH session after changing group membership
remote = chi.ssh.Remote(nodes[0])
remote.run("sudo groupadd docker; sudo usermod -aG docker $USER")
remote = chi.ssh.Remote(nodes[0])

groupadd: group 'docker' already exists


In [96]:
# copy docker configuration for private registry
remote.run("docker run -d -p 5000:5000 --restart always --name registry registry:2")
remote.run("sudo wget https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/daemon.json -O /etc/docker/daemon.json")
remote.run("sudo service docker restart")

Unable to find image 'registry:2' locally
2: Pulling from library/registry
8a49fdb3b6a5: Pulling fs layer
58116d8bf569: Pulling fs layer
4cb4a93be51c: Pulling fs layer
cbdeff65a266: Pulling fs layer
6b102b34ed3d: Pulling fs layer
6b102b34ed3d: Waiting
cbdeff65a266: Waiting
58116d8bf569: Verifying Checksum
58116d8bf569: Download complete
8a49fdb3b6a5: Verifying Checksum
8a49fdb3b6a5: Download complete
4cb4a93be51c: Verifying Checksum
4cb4a93be51c: Download complete
cbdeff65a266: Verifying Checksum
cbdeff65a266: Download complete
6b102b34ed3d: Verifying Checksum
6b102b34ed3d: Download complete
8a49fdb3b6a5: Pull complete
58116d8bf569: Pull complete
4cb4a93be51c: Pull complete
cbdeff65a266: Pull complete
6b102b34ed3d: Pull complete
Digest: sha256:20d084723c951e377e1a2a5b3df316173a845e300d57ccdd8ae3ab2da3439746
Status: Downloaded newer image for registry:2


596a30c0630b2bb865dd9f9978d521095ab90206bce5675a65d7e99fb1fa1d52


--2023-05-22 23:05:01--  https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/daemon.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48 [text/plain]
Saving to: ‘/etc/docker/daemon.json’

     0K                                                       100% 8.86M=0s

2023-05-22 23:05:01 (8.86 MB/s) - ‘/etc/docker/daemon.json’ saved [48/48]



<Result cmd='sudo service docker restart' exited=0>

#### Validate your setup

Use the following command to validate your Kubernetes cluster with 3 servers:

In [98]:
remote.run("kubectl get nodes")

NAME     STATUS   ROLES                  AGE     VERSION
node-0   Ready    control-plane,master   4m58s   v1.20.7
node-1   Ready    control-plane,master   4m36s   v1.20.7
node-2   Ready    <none>                 3m51s   v1.20.7


<Result cmd='kubectl get nodes' exited=0>

Also test the Docker install:

In [97]:
remote.run("docker run hello-world")

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
719385e32844: Pulling fs layer
719385e32844: Download complete
719385e32844: Pull complete
Digest: sha256:fc6cf906cbfa013e80938cdf0bb199fbdbb86d6e3e013783e5a766f50f5dbce0
Status: Downloaded newer image for hello-world:latest



Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



<Result cmd='docker run hello-world' exited=0>

# Delete resources

### Delete server

In [ ]:
if True: 
    for i in servers:
      chi.server.delete_server(i.id)

#### De-configure Network



In [ ]:
router_id = router['id']
subnet_id = subnet['id']

try:
    result = chi.network.remove_subnet_from_router(router_id, subnet_id)
except Exception as e:
    print(f"detach_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_router(router_id)
except Exception as e:
    print(f"delete_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_subnet(subnet_id)
except Exception as e:
    print(f"delete_subnet_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_network(network_id)
except Exception as e:
    print(f"delete_network_by_name error: {str(e)}")
    pass


#### Release Lease

In [ ]:
chi.lease.delete_lease(lease_id)